In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jaforero/baloto-colombia")

print("Path to dataset files:", path)

100%|██████████| 10.5k/10.5k [00:00<?, ?B/s]

Extracting files...
Path to dataset files: C:\Users\DAVID GOMEZ\.cache\kagglehub\datasets\jaforero\baloto-colombia\versions\57


In [ ]:
# Volver a leer el archivo subido recientemente y regenerar el PDF

# Leer el archivo
df = pd.read_csv("/mnt/data/baloto.csv")

# Limpieza rápida para permitir gráficos
df['FECHA_SORTEO'] = pd.to_datetime(df['FECHA_SORTEO'], errors='coerce')
df = df.drop_duplicates()
df = df.dropna()
df.columns = df.columns.str.strip().str.upper().str.replace(" ", "_")
columns_numeros = [col for col in df.columns if 'BALOTA' in col or 'NUMERO' in col]
df[columns_numeros] = df[columns_numeros].apply(pd.to_numeric, errors='coerce')
for col in columns_numeros:
    df = df[(df[col] >= 1) & (df[col] <= 45)]

# Porcentaje de aparición
numeros = df[columns_numeros].values.flatten()
frecuencia = pd.Series(numeros).value_counts().sort_index()
porcentaje = (frecuencia / frecuencia.sum()) * 100

# Pares e impares
def contar_pares_impares(fila):
    numeros = fila[columns_numeros]
    pares = sum(n % 2 == 0 for n in numeros)
    impares = len(numeros) - pares
    return pd.Series({'PARES': pares, 'IMPARES': impares})

df[['PARES', 'IMPARES']] = df.apply(contar_pares_impares, axis=1)

# Números consecutivos
def contar_consecutivos(fila):
    numeros = sorted(fila[columns_numeros])
    consecutivos = sum(1 for i in range(len(numeros)-1) if numeros[i+1] - numeros[i] == 1)
    return consecutivos

df['CONSECUTIVOS'] = df.apply(contar_consecutivos, axis=1)

# Rangos numéricos
def clasificar_rangos(num):
    if num <= 15:
        return "1-15"
    elif num <= 30:
        return "16-30"
    else:
        return "31-45"

rango_series = pd.Series(numeros).dropna().astype(int).map(clasificar_rangos)
rango_frecuencia = rango_series.value_counts(normalize=True) * 100

# Crear el PDF
pdf_path = "/mnt/data/Resumen_Proyecto_Baloto.pdf"
pdf = PdfPages(pdf_path)

# Gráfico 1: Porcentaje de aparición
plt.figure(figsize=(12, 6))
sns.barplot(x=porcentaje.index, y=porcentaje.values, palette="mako")
plt.title("Porcentaje de aparición de cada número del Baloto")
plt.xlabel("Número")
plt.ylabel("Porcentaje (%)")
plt.xticks(rotation=90)
pdf.savefig()
plt.close()

# Gráfico 2: Distribución de pares
plt.figure(figsize=(10, 5))
sns.histplot(df['PARES'], bins=range(0, 7), kde=False, color='skyblue', label='Pares')
plt.title("Distribución de cantidad de números pares por sorteo")
plt.xlabel("Cantidad de pares")
plt.ylabel("Número de sorteos")
plt.legend()
pdf.savefig()
plt.close()

# Gráfico 3: Números consecutivos
plt.figure(figsize=(10, 5))
sns.countplot(x='CONSECUTIVOS', data=df, palette='coolwarm')
plt.title("Cantidad de números consecutivos por sorteo")
plt.xlabel("Números consecutivos")
plt.ylabel("Número de sorteos")
pdf.savefig()
plt.close()

# Gráfico 4: Rangos numéricos
plt.figure(figsize=(8, 5))
rango_frecuencia.plot(kind='bar', color='coral')
plt.title("Porcentaje de aparición por rango numérico")
plt.ylabel("Porcentaje (%)")
plt.xlabel("Rango")
pdf.savefig()
plt.close()

# Página de conclusiones
fig = plt.figure(figsize=(11, 8.5))
plt.axis('off')
conclusiones = """
✅ CONCLUSIONES DEL ANÁLISIS DEL BALOTO

• Se aplicaron procesos de limpieza de datos: fechas, tipos numéricos, nulos y validación de rangos.

• El análisis de porcentajes muestra que ciertos números han aparecido con mayor frecuencia histórica.

• Se identificaron diferencias en la distribución de números pares e impares.

• La mayoría de los sorteos contienen entre 1 y 2 números consecutivos.

• Se analizaron rangos de números (1–15, 16–30, 31–45) mostrando su proporción en la historia del Baloto.

• Este tipo de análisis ayuda a visualizar patrones históricos, aunque no garantiza predicciones futuras.
"""
plt.text(0.05, 0.5, conclusiones, fontsize=12, va="center")
pdf.savefig(fig)
plt.close()

pdf.close()

pdf_path
